# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

## Setup

In [12]:
USE_OPENAI = False  # True

In [13]:
import os
import nest_asyncio

nest_asyncio.apply()

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [14]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["../data/metagpt.pdf"]).load_data()

## Define LLM and Embedding model

In [15]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [16]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

if USE_OPENAI:
    Settings.llm = OpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
else:
    Settings.llm = Ollama(model="llama3:instruct", request_timeout=120.0)
    Settings.embed_model = OllamaEmbedding(
        model_name="llama3:instruct",
        base_url="http://localhost:11434",
        ollama_additional_kwargs={"mirostat": 0})

Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

## Define Summary Index and Vector Index over the Same Data

In [17]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [18]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [19]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [20]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [21]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: Useful for summarization questions related to MetaGPT.
The document introduces MetaGPT, a meta-programming framework that enhances multi-agent systems based on Large Language Models (LLMs) through role specialization, workflow management, and efficient communication mechanisms. It incorporates an executable feedback mechanism to improve code generation quality during runtime. Extensive experiments demonstrate MetaGPT's superiority over previous approaches in software development tasks. The document also details the development process of a software application called the "Drawing App" using MetaGPT, outlining the roles of various agents and discussing the use of Python libraries for GUI creation and image handling. Additionally, it addresses the performance of MetaGPT in generating executable code, the importance of structured messaging in code generation, and the limitations and ethical concerns associated with the framework.


In [22]:
print(len(response.source_nodes))

34


In [23]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 0: Agents sharing information with other agents can be summarized in relation to MetaGPT.
Agents share information with other agents through a shared message pool and a subscription mechanism. The shared message pool allows agents to exchange structured messages directly, while the subscription mechanism enables agents to filter out irrelevant contexts and subscribe to relevant messages based on their profiles. This approach ensures efficient communication and information exchange among the agents, particularly in scenarios like software design and standard operating procedures where effective communication is crucial.


In [24]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: Ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results indicate that MetaGPT addresses challenges related to context utilization efficiently. It focuses on unfolding short natural language descriptions accurately to eliminate ambiguity and maintaining information validity in lengthy contexts. This approach enables Language Model Models (LLMs) to concentrate on relevant data without distraction, reducing hallucinations in software generation. By emphasizing granular tasks like requirement analysis and package selection, MetaGPT provides guided thinking that LLMs often lack in broad task solving.
